# Trying to get over 97% accuracy on MNIST

In [ ]:
%pip install keras-tuner
%pip install tensorboard-plugin-profile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 72.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import keras_tuner as kt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from pathlib import Path

# Load MNIST dataset

In [ ]:
mnist = tf.keras.datasets.mnist.load_data()

(x_train, y_train), (x_test, y_test) = mnist

# Scale down pixel intensities
x_train, x_test = x_train / 255.0, x_test / 255.0

print(x_train.shape) # Look at data

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
(60000, 28, 28)


# Create Model

In [ ]:
def build_mnist_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=4, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=512)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=5e-1, sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])

    if optimizer == "sgd":
      optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
      optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

    for _ in range(n_hidden):
      model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

    model.add(tf.keras.layers.Dense(10, activation="softmax"))

    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

    return model


class MnistHyperModel(kt.HyperModel):
  def build(self, hp):
    return build_mnist_model(hp)

  def fit(self, hp, model, X, y, **kwargs):
    if hp.Boolean("normalize"):
      norm_layer = tf.keras.layers.Normalization()
      X = norm_layer(X)
    return model.fit(X, y, **kwargs)


# Train Model

## Do Hyperparameter Tuning

In [ ]:
mnist_hyperband_tuner = kt.Hyperband(
    MnistHyperModel(),
    objective="val_accuracy",
    max_epochs=10,
    factor=3,
    directory="mnist_hyperband",
    project_name="hyperband_tuning")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
mnist_root_logdir = Path(mnist_hyperband_tuner.project_dir) / "tensorboard"
mnist_tensorboard_cb = tf.keras.callbacks.TensorBoard(mnist_root_logdir) # Allows for TensorBoard visualization
mnist_early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2) # Interrupts training when no progress on validation set for 2 trials
mnist_hyperband_tuner.search(x_train, y_train, epochs=10, validation_split=0.2,
                             callbacks=[mnist_early_stopping_cb, mnist_tensorboard_cb])

Trial 30 Complete [00h 01m 22s]
val_accuracy: 0.9577500224113464

Best val_accuracy So Far: 0.9760000109672546
Total elapsed time: 00h 15m 49s


## Get best model

In [ ]:
top_3_mnist_models = mnist_hyperband_tuner.get_best_models(num_models=3)
best_mnist_model = top_3_mnist_models[0]

best_fashion_trial = mnist_hyperband_tuner.oracle.get_best_trials(num_trials=1)[0]
best_fashion_trial.summary()

Trial 0028 summary
Hyperparameters:
n_hidden: 2
n_neurons: 163
learning_rate: 0.05067006455071722
optimizer: sgd
normalize: False
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9760000109672546


# Evaluate model

In [ ]:
best_mnist_model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9703 - loss: 0.0891


[0.07431023567914963, 0.9757000207901001]

# Save Model

In [ ]:
best_mnist_model.save('mnist_high_accuracy_model.keras')